# AGUME KENNETH B30309 S24B38/017

## SECTION A

## Q1. CSV vs Parquet (Core)

 How CSV stores data?

- In my experiment, the CSV file size was 32.86 MB while the Parquet file was only 7.03 MB. CSV stores data in plain text format, where each row is written sequentially and values are separated by commas. This means numbers and categories are stored as readable text which increases file size and requires more disk reading.

How Parquet stores data?

- Parquet, on the other hand, stores data in a columnar format. Each column is stored separately and compressed efficiently. This is because similar data types are stored together thus making compression  more effective. This explains why the Parquet file was significantly smaller.

What this means for disk I/O?

- The CSV loaded slightly fastee ir 1.09s vs 1.51s, Parquet reduces disk I/O overall because less data needs to be physically read from disk. As data grows larger, reduced disk I/O becomes more important than small decompression overhead.

## Q2. Column Selection Experiment

Why is this possible with Parquet but not efficient with CSV?

- When I loaded only two columns from the Parquet file, the load time dropped to 0.09 seconds because Parquet stores data column-by-column thus allowing the system to read only the required columns from disk.

Relate your answer to columnar storage design.

- CSV files store data row-by-row even when only two columns are needed, the entire row must be read and parsed thus making selective column reading inefficient in CSV. My experiment clearly demonstrated that columnar storage improves performance when working with analytical queries.

## Q3. Storage as a Bottleneck

- Loading the full dataset increased memory usage from 90 MB to 127 MB. Even though the dataset was only 32.86 MB on disk, reading it required additional memory and parsing time.

- As datasets increase in size, disk reading becomes a major bottleneck because data must first be transferred from storage into memory before processing. Disk I/O is slower than CPU operations therefore, even before CPU becomes overloaded, the speed of reading data from disk limits performance. This is why Big Data systems optimize storage formats and reduce unnecessary reads.

## SECTION B

## Q4. Full Load vs Chunk Processing

- Loading the entire CSV file at once increased memory usage and required the system to hold all rows in RAM simultaneously.
- While this worked for 500,000 rows, it would not scale well for millions or billions of rows.